In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from lightgbm import LGBMRegressor

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MLF Files/sales1CA2_Melt_Calendar_OutTreat_Date.csv')

<ipython-input-4-012c6dd136c0>:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MLF Files/sales1CA2_Melt_Calendar_OutTreat_Date.csv')


In [5]:
df['date'] = pd.to_datetime(df['date'])
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

for col in ['item_id', 'dept_id', 'store_id', 'event_name_1', 'event_type_1']:
    df[col] = df[col].astype('category')

df.dtypes

date            datetime64[ns]
item_id               category
dept_id               category
store_id              category
event_name_1          category
event_type_1          category
snap_CA                  int64
sales                    int64
year                     int64
month                    int64
day                      int64
dtype: object

In [6]:
X = df.drop(['sales', 'date'], axis=1)
y = df['sales']

In [7]:
# Train-Test Split (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Model Training with categorical features
categorical_features = ['item_id', 'dept_id', 'store_id', 'event_name_1', 'event_type_1']
model = LGBMRegressor()

In [8]:
model.fit(X_train, y_train, categorical_feature=categorical_features)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.190036 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1135
[LightGBM] [Info] Number of data points in the train set: 6409315, number of used features: 9
[LightGBM] [Info] Start training from score 0.687709


LGBMRegressor()

In [10]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE: {rmse}")

# Calculate R-squared
r2 = r2_score(y_test, y_pred)
print(f"R-squared (R²): {r2}")

RMSE: 1.3368541428007554
R-squared (R²): 0.44276926942333594


In [11]:
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
print("CV RMSE:", np.sqrt(-cv_scores).mean())

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.216005 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1135
[LightGBM] [Info] Number of data points in the train set: 6409315, number of used features: 9
[LightGBM] [Info] Start training from score 0.795441
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.222450 seconds.
You can set `force_row_wise

In [14]:
from sklearn.model_selection import GridSearchCV
grid_params = {
    'num_leaves': [25, 40],  # Finer increments around promising values
    'learning_rate': [0.05, 0.1],  # Logarithmic scaling for learning rate
    'n_estimators': [80, 120],  # Expanded range to explore more thoroughly
}
grid_search = GridSearchCV(model, grid_params, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.164503 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1135
[LightGBM] [Info] Number of data points in the train set: 5127452, number of used features: 9
[LightGBM] [Info] Start training from score 0.731909
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.158849 seconds.
You can set `force_row_wise

In [15]:
from sklearn.model_selection import GridSearchCV
grid_params = {
    'num_leaves': [40, 45],  # Finer increments around promising values
    'learning_rate': [0.1, 0.15],  # Logarithmic scaling for learning rate
    'n_estimators': [120, 150],  # Expanded range to explore more thoroughly
}
grid_search = GridSearchCV(model, grid_params, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.163841 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1135
[LightGBM] [Info] Number of data points in the train set: 5127452, number of used features: 9
[LightGBM] [Info] Start training from score 0.731909
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.292189 seconds.
You can set `force_row_wise

In [17]:
# Initialize the model with the best parameters from GridSearchCV
model = LGBMRegressor(learning_rate=0.15, n_estimators=150, num_leaves=45)

# Now, you can fit this model on your training data
model.fit(X_train, y_train, categorical_feature=categorical_features)

# Optionally, evaluate the model on your test data
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE: {rmse}")

# Calculate R-squared
r2 = r2_score(y_test, y_pred)
print(f"R-squared (R²): {r2}")

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.212983 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1135
[LightGBM] [Info] Number of data points in the train set: 6409315, number of used features: 9
[LightGBM] [Info] Start training from score 0.687709
RMSE: 1.3365693502545586
R-squared (R²): 0.44300665995074817


In [ ]:
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [ ]:
from lightgbm import plot_importance
import matplotlib.pyplot as plt

plot_importance(model, max_num_features=10, importance_type='gain')
plt.show()

In [ ]:
# Plotting the actual vs predicted values for the training dataset
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
plt.scatter(y_train, y_train_pred, color='blue')
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'k--', lw=4)
plt.title('Actual vs. Predicted (Train)')
plt.xlabel('True values of the Response Variable (Train)')
plt.ylabel('Predicted values of the Response Variable (Train)')

# Plotting the actual vs predicted values for the testing dataset
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_test_pred, color='red')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
plt.title('Actual vs. Predicted (Test)')
plt.xlabel('True values of the Response Variable (Test)')
plt.ylabel('Predicted values of the Response Variable (Test)')

# Show the plots
plt.show()